# meassurement of the accuracy 

accuracy is 
mean{ p in dotplot @ i,j  |  i,j in real_structure }

In [1]:
import sys
sys.path.insert(0,'..')
sys.path.insert(0,'/home/ikea/miniconda2/lib/python2.7/site-packages')


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)
import sys
sys.path.insert(0,'..')
from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

# Load data

In [3]:
import rna_tools.rna_io as rio

def getdata():
    return rio.get_all_data("../data/RNA16.react","../data/RNA16.dbn")  # {key: rea, seq, stru}

data = getdata()

for k,v in data.items():print k,"\t",  len(v[1])

data for 'tRNA-asp' is corrupted, ignoring...
ADDRSW 	121
GLYCFN 	205
tRNAphe 	76
RNAsep 	154
ZHCV 	95
Z-CIDGMP-1 	135
adenin 	71
23sRNA 	2904
p564 	155
srRNA 	120
ZGLY 	198
MDLOOP 	80
R009 	1542
5srRNA 	170
TRP5 	202


# train predict

In [4]:
import simushape as sim
from rna_tools.rnaplfold import rnaplfold

data = getdata()
model  = sim.make_model(data,['23sRNA','R009'],sim.make_forestregressor())
data.pop("23sRNA")
data.pop("R009")
keys = data.keys()

def get_str(seq):
    for _, data_seq, db in data.values():
        if seq == data_seq:
            return [db]
    print "SHIT"

# acc_calculator needs seq struct rea
predictions = { k: (data[k][1],data[k][2],sim.predict(model,data[k][1], seq_to_db_function=get_str)) for k in keys  }

shaker_plain = { k: (data[k][1],data[k][2],sim.predict(model,data[k][1])) for k in keys  }


data for 'tRNA-asp' is corrupted, ignoring...


# get accuracy

In [5]:
from rna_tools.sukosd import sukosd
from rna_tools.rna_accuracy import get_structure_accuracy

acc_shaker = {k: v
              (*predictions[k])  for  k in keys} 
acc_shaker_plain = {k: get_structure_accuracy(*shaker_plain[k]) for  k in keys} 
predictions_suko = { k: (data[k][1],data[k][2],sukosd(data[k][2])) for k in keys }
acc_suko = {k: get_structure_accuracy(*predictions_suko[k])  for  k in keys} 
acc_real = {k: get_structure_accuracy(data[k][1],data[k][2],data[k][0])  for  k in keys}
acc_noshape = {k: get_structure_accuracy(data[k][1],data[k][2],None)  for  k in keys}




../rna_tools/rnaplfold.py:31: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if react == None:


# Visualize 

In [6]:
import pandas
import numpy as np
index= ['suko',"shaker","shaker_plain",'real',"no_shape"]
data = map(lambda x: [x[k] for k in keys] ,[acc_suko, acc_shaker,acc_shaker_plain, acc_real, acc_noshape])
map(lambda x:x.append(np.mean(x)),data)
df = pandas.DataFrame(data, columns=keys+['mean'], index=index).T

print df

                suko    shaker  shaker_plain      real  no_shape
ADDRSW      0.975322  0.993288      0.993241  0.987444  0.872423
GLYCFN      0.851236  0.972468      0.661192  0.000000  0.565605
tRNAphe     0.973463  0.994329      0.986253  0.995362  0.504352
RNAsep      0.689783  0.696040      0.683036  0.676623  0.530544
ZHCV        0.854657  0.897167      0.897715  0.533042  0.744884
Z-CIDGMP-1  0.774839  0.913900      0.746818  0.759976  0.727020
adenin      0.998081  0.990202      0.990312  0.981553  0.871697
p564        0.893542  0.859896      0.447638  0.904553  0.625162
srRNA       0.881829  0.906264      0.277575  0.849606  0.274786
ZGLY        0.936920  0.968255      0.422640  0.862487  0.482860
MDLOOP      0.999660  0.998307      0.998486  0.991718  0.988975
5srRNA      0.936069  0.969630      0.259991  0.252744  0.260634
TRP5        0.920135  0.909863      0.899801  0.821419  0.838098
mean        0.898887  0.928431      0.712669  0.739733  0.637465


In [7]:
print df.to_latex()

\begin{tabular}{lrrrrr}
\toprule
{} &      suko &    shaker &  shaker\_plain &      real &  no\_shape \\
\midrule
ADDRSW     &  0.975322 &  0.993288 &      0.993241 &  0.987444 &  0.872423 \\
GLYCFN     &  0.851236 &  0.972468 &      0.661192 &  0.000000 &  0.565605 \\
tRNAphe    &  0.973463 &  0.994329 &      0.986253 &  0.995362 &  0.504352 \\
RNAsep     &  0.689783 &  0.696040 &      0.683036 &  0.676623 &  0.530544 \\
ZHCV       &  0.854657 &  0.897167 &      0.897715 &  0.533042 &  0.744884 \\
Z-CIDGMP-1 &  0.774839 &  0.913900 &      0.746818 &  0.759976 &  0.727020 \\
adenin     &  0.998081 &  0.990202 &      0.990312 &  0.981553 &  0.871697 \\
p564       &  0.893542 &  0.859896 &      0.447638 &  0.904553 &  0.625162 \\
srRNA      &  0.881829 &  0.906264 &      0.277575 &  0.849606 &  0.274786 \\
ZGLY       &  0.936920 &  0.968255 &      0.422640 &  0.862487 &  0.482860 \\
MDLOOP     &  0.999660 &  0.998307 &      0.998486 &  0.991718 &  0.988975 \\
5srRNA     &  0.936069 &  0.

In [12]:
from tabulate import tabulate 
print tabulate(df, headers="keys", tablefmt="orgtbl")

|            |     suko |   shaker |   shaker_plain |     real |   no_shape |
|------------+----------+----------+----------------+----------+------------|
| ADDRSW     | 0.975322 | 0.993288 |       0.993241 | 0.987444 |   0.872423 |
| GLYCFN     | 0.851236 | 0.972468 |       0.661192 | 0        |   0.565605 |
| tRNAphe    | 0.973463 | 0.994329 |       0.986253 | 0.995362 |   0.504352 |
| RNAsep     | 0.689783 | 0.69604  |       0.683036 | 0.676623 |   0.530544 |
| ZHCV       | 0.854657 | 0.897167 |       0.897715 | 0.533042 |   0.744884 |
| Z-CIDGMP-1 | 0.774839 | 0.9139   |       0.746818 | 0.759976 |   0.72702  |
| adenin     | 0.998081 | 0.990202 |       0.990312 | 0.981553 |   0.871697 |
| p564       | 0.893542 | 0.859896 |       0.447638 | 0.904553 |   0.625162 |
| srRNA      | 0.881829 | 0.906264 |       0.277575 | 0.849606 |   0.274786 |
| ZGLY       | 0.93692  | 0.968255 |       0.42264  | 0.862487 |   0.48286  |
| MDLOOP     | 0.99966  | 0.998307 |       0.998486 | 0.991718 |